# SAL Script - Slew and Track

Performs a single slew and track using SAL Script.  
It can be used as a minimum functionality prototype for the Soak Test.

## Notebook Setup

Start by importing relevat libraries, setting up domain, remotes, logs, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import yaml

import numpy as np

from astropy import units as u

from lsst.ts import salobj
from lsst.ts.standardscripts.maintel.track_target import TrackTarget

In [ ]:
track_target = TrackTarget(index=1)
await track_target.start_task

In [ ]:
mtcs = track_target.tcs

## Prepare Components

In [ ]:
await mtcs.set_state(salobj.State.DISABLED, components=["mtmount", "mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount", "mtrotator"])

In [ ]:
az = mtcs.rem.mtmount.tel_azimuth.get().actualPosition
el = mtcs.rem.mtmount.tel_elevation.get().actualPosition

print(az, el)

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=2, elevation=62)

In [ ]:
await mtcs.rem.mtmount.cmd_homeBothAxes.start(timeout=300)

## Emulate ScriptQueue

In [ ]:
radec = track_target.tcs.radec_from_azel(60., 90.)

In [ ]:
configuration = yaml.safe_dump(
    {
        "slew_icrs": {
            "ra": float(radec.ra.hour),
            "dec": float(radec.dec.deg),
        }, 
        "rot_value": 0., 
        "rot_type": "Physical", 
        "track_for": 120., 
        "stop_when_done": True,
        "ignore": [
            "mtaos",
            "mtdome",
            "mtdometrajectory",
            "mthexapod_1",
            "mthexapod_2", 
            "mtm1m3", 
            "mtm2",
        ],
    }
)

print(configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
track_target.set_state(ScriptState.UNCONFIGURED)

# Configure the script, which puts the ScriptState to CONFIGURED
config_data = track_target.cmd_configure.DataType()
config_data.config = configuration

await track_target.do_configure(config_data)

In [ ]:
print("Start:", Time.now())
results = await track_target.run()
print("Done:", Time.now())